# 初始化

In [ ]:
from unsloth import FastLanguageModel
import torch
import re 
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth


# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

## 模型提示词以及加载数据集

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
alpaca_prompt = """
### Instruction:
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stability, anxiety levels, and stress coping abilities.

    ## Constraints
    - Evaluate based solely on the information provided by the user, ensuring the assessment is objective and accurate.
    - Even if the information provided by the user may be incomplete, You will still try your best to give the scores for the Big Five personality traits.


### Input:
{}
### Response:
{}"""
def formatting_prompts_func(examples):
    # instructions = examples["introduction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input_data, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input_data, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



In [ ]:
from datasets import load_dataset,DatasetDict
dataset = load_dataset("json", data_files="../data/results.jsonl")
dataset = dataset.map(formatting_prompts_func, batched = True,)
full_dataset = dataset["train"]  # 提取实际数据部分
# 直接划分：90% 训练，10% 测试
train_test = full_dataset.train_test_split(test_size=0.2, seed=42)
# 组合为 DatasetDict
final_dataset = DatasetDict({
    "train": train_test["train"],
    "test": train_test["test"],
})
# 查看划分结果
print(final_dataset)

# 调优前模型输出测试方法

In [ ]:
# prrompt = f"{final_dataset['test'][0]['introduction']}\n{final_dataset['test'][0]['input']}"
def test_basemodel(index):
    prompt = alpaca_prompt.format(final_dataset['test'][index]['input'],' ')
    prompt_encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **prompt_encoding,
        use_cache=True,
        max_new_tokens=128,
        temperature=0.7,
        top_k=50,
        top_p=0.9
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    print(generated_text[0])
    print("\n以上为模型输出，以下为测试集的真实输出：")
    print(final_dataset['test'][index]['output'])
   

# 训练调优模型

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_dataset['train'],
    dataset_text_field = "text",
    formatting_func=formatting_prompts_func,
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 80,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
# trainer_stats = trainer.train()
# # @title Show final memory and time stats
# 
# used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# used_percentage = round(used_memory / max_memory * 100, 3)
# lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
# print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
# print(
#     f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
# )
# print(f"Peak reserved memory = {used_memory} GB.")
# print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
# print(f"Peak reserved memory % of max memory = {used_percentage} %.")
# print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# 测试调优后模型

In [99]:

def check_result(index):
    alpaca_prompt = """
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stability, anxiety levels, and stress coping abilities.

    ## Constraints
    - Evaluate based solely on the information provided by the user, ensuring the assessment is objective and accurate.
    - Even if the information provided by the user may be incomplete, You will still try your best to give the scores for the Big Five personality traits.

### Input:
{}
### Response:
{}"""
    prompt = alpaca_prompt.format(final_dataset['test'][index]['input'], ' ')
    prompt_encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **prompt_encoding,
        use_cache=True,
        max_new_tokens=128,
        temperature=0.85,
        top_k=50,
        top_p=0.9
    )
    # 将生成的输出解码为文本
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # 打印生成的文本
    # print(generated_text[0])
    # print("以上为模型输出，以下为测试集的真实输出：")
    # print(final_dataset['test'][index]['output'])
    return(generated_text[0])

import re

def calculate_mae(ai_scores: dict, real_scores: dict) -> float:
    """计算两组大五人格特质分数之间的平均绝对误差(MAE)
    
    Args:
        ai_scores: 模型预测的特质分数字典，应包含全部五个特质
        real_scores: 真实标签的特质分数字典，应包含全部五个特质
        
    Returns:
        float: 平均绝对误差值
    """
    traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    total_diff = 0.0
    
    for trait in traits:
        total_diff += abs(ai_scores[trait] - real_scores[trait])
    
    return total_diff / len(traits)

def compute_mae_by_index(index: int) -> float:
    """根据测试集索引计算模型输出的MAE
    
    Args:
        index: 测试集样本索引
        
    Returns:
        float: 该样本的平均绝对误差值
    """
    ai_text = check_result(index)
    ai_scores = get_scores_from_text(ai_text)
    real_scores = get_scores_from_text(final_dataset['test'][index]['output'])
    
    if len(ai_scores) != 5 or len(real_scores) != 5:
        print(f"解析异常 - AI输出: {ai_text}\n真实输出: {final_dataset['test'][index]['output']}")
        return float('nan')
    
    return calculate_mae(ai_scores, real_scores)

def get_scores_from_text(text: str) -> dict:
    """从文本中提取大五人格特质分数
    
    Args:
        text: 包含特质分数的文本
        
    Returns:
        dict: 包含特质名称和对应分数的字典
    """
    traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    pattern = r"[-\s*]*\b({})\b[^:\n]*[:：]\s*(\d+\.\d+)"
    compiled_pattern = re.compile(pattern.format("|".join(traits)), re.IGNORECASE)
    
    traits_dict = {}
    for name, score in compiled_pattern.findall(text):
        normalized_name = name.strip().capitalize()
        if normalized_name in traits:
            traits_dict[normalized_name] = float(score)
    
    return traits_dict

    

## 详细测试示例

In [73]:
print(check_result(0))
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][0]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [74]:
print(check_result(1))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][1]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [75]:
print(check_result(2))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][2]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [76]:
print(check_result(3))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][3]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [77]:
print(check_result(4))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][4]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [78]:
print(check_result(5))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][5]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

# 计算模型准确性参数

In [97]:
test_mae = compute_mae_by_index(0)
print(f"{(1-test_mae)*100}%")

89.8%


In [100]:
import math
from tqdm import tqdm
total_mae = 0.0
valid_samples = 0

for idx in range(len(final_dataset['test'])):
    try:
        mae = compute_mae_by_index(idx)
        if not math.isnan(mae):
            total_mae += mae
            valid_samples += 1
    except Exception as e:
        print(f"计算索引 {idx} 时出错: {str(e)}")

final_mae = total_mae / valid_samples
print(f"测试集平均MAE: {final_mae:.4f}")
print(f"测试集正确率: {(1-final_mae)*100:.2f}%")

解析异常 - AI输出: 
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emoti

In [101]:
print(f"测试集平均MAE: {final_mae:.4f}")
print(f"测试集正确率: {(1-final_mae)*100:.2f}%")
print(f"测试集样本数: {valid_samples}")

测试集平均MAE: 0.1404
测试集正确率: 85.96%
测试集样本数: 54


In [102]:
from tqdm import tqdm
import math

total_mae = 0.0
valid_samples = 0

# 创建进度条对象
with tqdm(total=len(final_dataset['test']), 
         desc="评估进度",
         unit="样本",
         bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:

    for idx in range(len(final_dataset['test'])):
        try:
            mae = compute_mae_by_index(idx)
            if not math.isnan(mae):
                total_mae += mae
                valid_samples += 1
                
                # 实时更新进度条描述
                pbar.set_postfix({
                    '当前MAE': f"{mae:.4f}",
                    '平均MAE': f"{total_mae/valid_samples:.4f}",
                    '有效样本': valid_samples
                })
                
        except Exception as e:
            pbar.write(f"错误 @ 样本{idx}: {str(e)}")  # 使用pbar.write避免干扰进度条
        finally:
            pbar.update(1)  # 确保每次迭代都更新进度条

# 最终计算结果
if valid_samples > 0:
    final_mae = total_mae / valid_samples
    print(f"\n{'='*40}")
    print(f"最终测试集平均MAE: {final_mae:.4f}")
    print(f"理论最大正确率: {(1 - final_mae)*100:.2f}%")
    print(f"有效样本/总样本: {valid_samples}/{len(final_dataset['test'])}")
    print('='*40)
else:
    print("警告：没有有效样本可计算MAE")

评估进度:  25%|██▌       | 15/60 [00:39<01:55]

解析异常 - AI输出: 
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emoti

评估进度:  47%|████▋     | 28/60 [01:13<01:21]

解析异常 - AI输出: 
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emoti

评估进度:  93%|█████████▎| 56/60 [02:29<00:10]

解析异常 - AI输出: 
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emoti

评估进度: 100%|██████████| 60/60 [02:40<00:00]


最终测试集平均MAE: 0.1458
理论最大正确率: 85.42%
有效样本/总样本: 57/60


# 保存模型

In [ ]:
# model.save_pretrained("lora_model")  # Local saving
# tokenizer.save_pretrained("lora_model")